Import tensorflow

In [4]:
import tensorflow as tf

Reset deafault graph

In [5]:
tf.reset_default_graph()

Load mnist data

In [6]:
from tensorflow.examples.tutorials.mnist import input_data

In [7]:
mnist = input_data.read_data_sets('datasets/minst/')

Extracting datasets/minst/train-images-idx3-ubyte.gz
Extracting datasets/minst/train-labels-idx1-ubyte.gz
Extracting datasets/minst/t10k-images-idx3-ubyte.gz
Extracting datasets/minst/t10k-labels-idx1-ubyte.gz


In [8]:
X_val = mnist.validation.images
y_val = mnist.validation.labels

Define a NN of two hidden layers with Relu. 1st hidden layer has 300 neurons, and 2nd hidden layer has 50 neurons. Add dropout layer after each hidden layer.

In [9]:
n_input = 28**2
n_hidden1 = 300
n_hidden2 = 50
n_output = 10
dropout_rate = 0.5

In [10]:
X = tf.placeholder(dtype=tf.float32, shape=(None, n_input), name='X')
y = tf.placeholder(dtype=tf.int32, shape=(None,), name='y')

In [11]:
training = tf.Variable(initial_value=False, trainable=False)

In [12]:
with tf.name_scope('dnn'):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name='hidden1')
    hidden1_dropout = tf.layers.dropout(hidden1, rate=dropout_rate, training=training)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name='hidden2')
    hidden2_dropout = tf.layers.dropout(hidden2, rate=dropout_rate, training=training)
    logits = tf.layers.dense(hidden2, n_output, activation=None, name='logits')

In [13]:
with tf.name_scope('loss'):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(entropy, name='loss')

In [14]:
with tf.name_scope('train'):
    opt = tf.train.MomentumOptimizer(learning_rate=0.01, momentum=0.9)
    train_op = opt.minimize(loss, name='train_op')

In [15]:
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(predictions=logits, targets=y, k=1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='accuracy')
    accuracy_sum = tf.summary.scalar('accuracy', accuracy)

In [16]:
init = tf.global_variables_initializer()

In [17]:
file_writer_train = tf.summary.FileWriter('logs/dropout/train')
file_writer_val = tf.summary.FileWriter('logs/dropout/val')

In [18]:
n_epoch = 20
batch_size = 50
n_step = mnist.train.num_examples // batch_size

In [19]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epoch):
        for step in range(n_step):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(train_op, feed_dict = {X: X_batch, y: y_batch, training:True})
        accuracy_val, accuracy_val_sum = sess.run([accuracy, accuracy_sum], feed_dict={X: mnist.validation.images, y: mnist.validation.labels})
        accuracy_train, accuracy_train_sum = sess.run([accuracy, accuracy_sum], feed_dict={X:mnist.train.images, y:mnist.train.labels})
        global_step = n_step*(epoch+1)
        file_writer_train.add_summary(accuracy_train_sum, global_step=global_step)
        file_writer_val.add_summary(accuracy_val_sum, global_step=global_step)
        print('Epoch: {}, training accuracy: {:.4f}, validation accuracy: {:.4f}'.format(epoch, accuracy_train, accuracy_val))
    
    file_writer_train.close()
    file_writer_val.close()

Epoch: 0, training accuracy: 0.9576, validation accuracy: 0.9610
Epoch: 1, training accuracy: 0.9742, validation accuracy: 0.9700
Epoch: 2, training accuracy: 0.9759, validation accuracy: 0.9740
Epoch: 3, training accuracy: 0.9867, validation accuracy: 0.9790
Epoch: 4, training accuracy: 0.9899, validation accuracy: 0.9794
Epoch: 5, training accuracy: 0.9915, validation accuracy: 0.9810
Epoch: 6, training accuracy: 0.9912, validation accuracy: 0.9790
Epoch: 7, training accuracy: 0.9960, validation accuracy: 0.9810
Epoch: 8, training accuracy: 0.9965, validation accuracy: 0.9818
Epoch: 9, training accuracy: 0.9981, validation accuracy: 0.9824
Epoch: 10, training accuracy: 0.9989, validation accuracy: 0.9822
Epoch: 11, training accuracy: 0.9994, validation accuracy: 0.9834
Epoch: 12, training accuracy: 0.9996, validation accuracy: 0.9822
Epoch: 13, training accuracy: 0.9996, validation accuracy: 0.9826
Epoch: 14, training accuracy: 0.9999, validation accuracy: 0.9828
Epoch: 15, training 